# Circuit simulation examples

In [1]:
import obi_one as obi

<class 'entitysdk.models.morphology.BrainLocation'>

MULTIPLE VALUE PARAMETERS
timestamps.timestamps_1.interval: [1.0, 5.0]
stimuli.stimulus_1.spike_probability: [0.5, 0.8]
initialize.simulation_length: [100.0, 500.0]

COORDINATE PARAMETERS
timestamps.timestamps_1.interval: 1.0, stimuli.stimulus_1.spike_probability: 0.5, initialize.simulation_length: 100.0
timestamps.timestamps_1.interval: 1.0, stimuli.stimulus_1.spike_probability: 0.5, initialize.simulation_length: 500.0
timestamps.timestamps_1.interval: 1.0, stimuli.stimulus_1.spike_probability: 0.8, initialize.simulation_length: 100.0
timestamps.timestamps_1.interval: 1.0, stimuli.stimulus_1.spike_probability: 0.8, initialize.simulation_length: 500.0
timestamps.timestamps_1.interval: 5.0, stimuli.stimulus_1.spike_probability: 0.5, initialize.simulation_length: 100.0
timestamps.timestamps_1.interval: 5.0, stimuli.stimulus_1.spike_probability: 0.5, initialize.simulation_length: 500.0
timestamps.timestamps_1.interval: 5.0, stimuli.stim

### Loading two circuits

In [2]:
circuit = obi.Circuit(name="ToyCircuit-S1-6k", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'ToyCircuit-S1-6k' with 5924 neurons and 568717 synapses


In [3]:
# circuit2 = obi.Circuit(name="ToyCircuit-S1-6k-duplicate", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
circuit2 = obi.Circuit(name="nbS1-HEX0-beta", path="/Users/pokorny/Data/Circuits/nbS1-HEX0-beta/circuit_config.json")
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes.size} neurons and {circuit2.sonata_circuit.edges.size} synapses")

Circuit 'nbS1-HEX0-beta' with 39056 neurons and 48129135 synapses


### Defining the circuit simulation

In [4]:
intracellular_location_set_1 = obi.SectionIntracellularLocationSet(circuit=circuit, neuron_ids=(1, 2, 3), section='soma')
xyz_extracellular_location_set_1 = obi.XYZExtracellularLocationSet(circuit=circuit, xyz_locations=((0.9, 0.2, 0.3), (0.1, 0.1, 0.3)))

neuron_set_1 = obi.ExistingNeuronSet(name="L1Sample", node_set="Layer1", random_sample=[None, 10, 50])
id_synapse_set_1 = obi.IDSynapseSet(circuit=circuit, synapse_ids=(1, 2, 3))

timestamps_1 = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=1.0)
stimulus_1 = obi.SynchronousSingleSpikeStimulus(spike_probability=0.5, timestamps=timestamps_1, synapse_set=id_synapse_set_1)
recording_1 = obi.IntracellularLocationSetVoltageRecording(start_time=0.0, end_time=1.0, intracellular_location_set=intracellular_location_set_1)

simulations_initialize = obi.SimulationsForm.Initialize(
    circuit=[circuit, circuit2],
    node_set=neuron_set_1,
    simulation_length=100
)

"""
Fill form with Blocks
"""
simulations_form = obi.SimulationsForm(
                                    initialize=simulations_initialize,

                                    intracellular_location_sets={"section_intracellular_location_set_1": section_intracellular_location_set_1},
                                    extracellular_location_sets={"xyz_extracellular_location_set_1": xyz_extracellular_location_set_1},

                                    neuron_sets={neuron_set_1.name: neuron_set_1},
                                    synapse_sets={"id_synapse_set_1": id_synapse_set_1},

                                    timestamps={"timestamps_1": timestamps_1}, 
                                    stimuli={"stimulus_1": stimulus_1}, 
                                    recordings={"recording_1": recording_1},
                                    )

In [5]:
grid_scan = obi.GridScan(form=simulations_form, output_root='../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.coordinate_instances(display=True)
grid_scan.generate()

In [6]:
# Deserialization
# grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../obi-output/circuit_simulations/grid_scan/generate_scan_config.json")